# Introdução

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplleaflet as mpl
import random
import folium
import seaborn as sns
from tqdm.notebook import tqdm
%matplotlib inline

# Dataset

In [1]:
#selecionando a tabela de New York City
filename = 'dataset_TSMC2014_NYC.csv'

In [10]:
#Convertendo os dados tipo UTC para data/hora
#%%time
df = pd.read_csv(filename, parse_dates=['utcTimestamp'])
df.head()

,userId,venueId,venueCategoryId,venueCategory,latitude,longitude,timezoneOffset,utcTimestamp
0,470,49bbd6c0f964a520f4531fe3,4bf58dd8d48988d127951735,Arts & Crafts Store,40.719810,-74.002581,-240,2012-04-03 18:00:09+00:00
1,979,4a43c0aef964a520c6a61fe3,4bf58dd8d48988d1df941735,Bridge,40.606800,-74.044170,-240,2012-04-03 18:00:25+00:00
2,69,4c5cc7b485a1e21e00d35711,4bf58dd8d48988d103941735,Home (private),40.716162,-73.883070,-240,2012-04-03 18:02:24+00:00
3,395,4bc7086715a7ef3bef9878da,4bf58dd8d48988d104941735,Medical Center,40.745164,-73.982519,-240,2012-04-03 18:02:41+00:00
4,87,4cf2c5321d18a143951b5cec,4bf58dd8d48988d1cb941735,Food Truck,40.740104,-73.989658,-240,2012-04-03 18:03:00+00:00


In [13]:
df.shape

(227428, 8)

### Vamos trabalhar com 20 registros apenas

In [18]:
#Número de usuários
df['userId'].nunique()

1083

In [25]:
#selecionado aleatóriamente 20 ids de usuários
users = random.sample(list(df['userId'].unique()),20)
print(users)

[386, 856, 512, 371, 675, 218, 308, 1004, 980, 1075, 822, 725, 40, 830, 466, 787, 1032, 278, 94, 450]


In [28]:
#selecionando todos usuários que não estão na lista aleatória
user_to_drop = df[df['userId'].isin(users)==False].index
print(user_to_drop)

Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            227418, 227419, 227420, 227421, 227422, 227423, 227424, 227425,
            227426, 227427],
           dtype='int64', length=223160)


In [29]:
#dropando os usuários que não estão na lista
df.drop(user_to_drop, inplace=True)

In [30]:
#Conferindo se o dataset foi reduzido
df['userId'].nunique()

20

### Ordenação dos dados <br>
Ao trabalhar com dados de trajetória, é recomendando sempre ordenar para não perder a sequencia dos acontecimentos

In [32]:
#Antes da ordenação
df.head()

,userId,venueId,venueCategoryId,venueCategory,latitude,longitude,timezoneOffset,utcTimestamp
35,1032,4bf5cedf4d5f20a1833d98fe,4bf58dd8d48988d1f9931735,Road,40.901058,-74.150763,-240,2012-04-03 18:18:54+00:00
52,1032,4e6657e7d4c06542ac9acee2,4bf58dd8d48988d177941735,Medical Center,40.885440,-74.138870,-240,2012-04-03 18:32:27+00:00
163,725,4b150df9f964a52011a823e3,4bf58dd8d48988d124941735,Office,40.749142,-73.975051,-240,2012-04-03 19:51:41+00:00
286,371,4e6d90f02271a8cabfdbfb27,4bf58dd8d48988d1f9931735,Road,40.784864,-73.786329,-240,2012-04-03 21:12:05+00:00
287,371,4c60606413791b8dc5624faf,4bf58dd8d48988d1f9931735,Road,40.786848,-73.789104,-240,2012-04-03 21:12:23+00:00


In [34]:
#Após a ordenação
df.sort_values(['userId', 'utcTimestamp'], inplace=True)
df.head()

,userId,venueId,venueCategoryId,venueCategory,latitude,longitude,timezoneOffset,utcTimestamp
1328,40,4ca3b7a67f84224bf834c758,4bf58dd8d48988d16a941735,Bakery,40.760075,-73.826976,-240,2012-04-04 11:41:26+00:00
3443,40,4c51e802d797e21e020ed67c,4bf58dd8d48988d1b2941735,College Academic Building,40.770282,-73.735549,-240,2012-04-07 15:49:09+00:00
5501,40,4b0c82edf964a520953e23e3,4bf58dd8d48988d149941735,Thai Restaurant,40.741183,-73.881230,-240,2012-04-08 17:42:07+00:00
5670,40,4f81ded6e4b09a147825fde9,4bf58dd8d48988d110951735,Salon / Barbershop,40.746021,-73.918332,-240,2012-04-08 18:54:23+00:00
5694,40,4a6a0dbcf964a5207bcc1fe3,4bf58dd8d48988d112951735,Hardware Store,40.752394,-73.912969,-240,2012-04-08 19:08:17+00:00


### Formato dos dados

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4268 entries, 1328 to 222934
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   userId           4268 non-null   int64              
 1   venueId          4268 non-null   object             
 2   venueCategoryId  4268 non-null   object             
 3   venueCategory    4268 non-null   object             
 4   latitude         4268 non-null   float64            
 5   longitude        4268 non-null   float64            
 6   timezoneOffset   4268 non-null   int64              
 7   utcTimestamp     4268 non-null   datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), float64(2), int64(2), object(3)
memory usage: 460.1+ KB


In [37]:
#Para que essa coluna de dados seja considerada limpa, precisamos remover o "+"
df['utcTimestamp']

1328     2012-04-04 11:41:26+00:00
3443     2012-04-07 15:49:09+00:00
5501     2012-04-08 17:42:07+00:00
5670     2012-04-08 18:54:23+00:00
5694     2012-04-08 19:08:17+00:00
                    ...           
212935   2013-01-15 23:05:50+00:00
220781   2013-02-02 05:37:56+00:00
220978   2013-02-02 16:29:15+00:00
221248   2013-02-02 22:22:09+00:00
222934   2013-02-05 23:32:46+00:00
Name: utcTimestamp, Length: 4268, dtype: datetime64[ns, UTC]

In [39]:
#Colocando variavel na forma correta
df['utcTimestamp'] = df['utcTimestamp'].dt.tz_localize(None)
df['utcTimestamp']

1328     2012-04-04 11:41:26
3443     2012-04-07 15:49:09
5501     2012-04-08 17:42:07
5670     2012-04-08 18:54:23
5694     2012-04-08 19:08:17
                 ...        
212935   2013-01-15 23:05:50
220781   2013-02-02 05:37:56
220978   2013-02-02 16:29:15
221248   2013-02-02 22:22:09
222934   2013-02-05 23:32:46
Name: utcTimestamp, Length: 4268, dtype: datetime64[ns]

In [40]:
#transformando dados categoricos
df['userId'] = pd.Categorical(df['userId'])
df['venueId'] = pd.Categorical(df['venueId'])
df['venueCategoryId'] = pd.Categorical(df['venueCategoryId'])
df['venueCategory'] = pd.Categorical(df['venueCategory'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4268 entries, 1328 to 222934
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   userId           4268 non-null   category      
 1   venueId          4268 non-null   category      
 2   venueCategoryId  4268 non-null   category      
 3   venueCategory    4268 non-null   category      
 4   latitude         4268 non-null   float64       
 5   longitude        4268 non-null   float64       
 6   timezoneOffset   4268 non-null   int64         
 7   utcTimestamp     4268 non-null   datetime64[ns]
dtypes: category(4), datetime64[ns](1), float64(2), int64(1)
memory usage: 427.3 KB


In [41]:
#Descrição de dados
df.describe(include='all', datetime_is_numeric=True)

,userId,venueId,venueCategoryId,venueCategory,latitude,longitude,timezoneOffset,utcTimestamp
count,4268.0,4268,4268,4268,4268.000000,4268.000000,4268.000000,4268
unique,20.0,1570,250,175,NaN,NaN,NaN,NaN
top,371.0,4cd1e59fde0f6dcbf7e67363,4bf58dd8d48988d103941735,Home (private),NaN,NaN,NaN,NaN
freq,702.0,104,640,640,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,40.753622,-73.959803,-255.365511,2012-07-29 19:47:48.092312064
min,NaN,NaN,NaN,NaN,40.571621,-74.223850,-300.000000,2012-04-03 18:18:54
25%,NaN,NaN,NaN,NaN,40.721352,-73.998859,-300.000000,2012-05-11 18:55:59.249999872
50%,NaN,NaN,NaN,NaN,40.757823,-73.972626,-240.000000,2012-06-11 14:37:28
75%,NaN,NaN,NaN,NaN,40.781685,-73.922552,-240.000000,2012-11-06 22:46:36
max,NaN,NaN,NaN,NaN,40.982940,-73.686199,-240.000000,2013-02-14 00:17:29
